In [ ]:
__author__ = 'Benjamin Alan Weaver <baweaver@lbl.gov>, Alice Jacques <alice.jacques@noao.edu>'
__version__ = '20200917'
__datasets__ = ['sdss_dr16']
__keywords__ = ['spectroscopy', 'fits', 'plot:animation']

# How to Visualize SDSS Spectroscopic Data with Bokeh

*Benjamin Alan Weaver & NOIRLab Astro Data Lab Team*

### Table of contents

* [Goals & Notebook Summary](#Goals)
* [Disclaimer & Attribution](#Disclaimer-&-Attribution)
* [Imports & Setup](#Imports-&-Setup)
* [Loading Spectra](#Loading-Spectra)
* [Plotting](#Plotting)
* [Visual inspection results](#Visual-inspection-results)
* [Navigating through targets](#Navigating-through-targets)

## Goals

Display and interact with SDSS spectroscopic data.

## Notebook Summary

[Bokeh](https://bokeh.pydata.org/en/latest/) is a library for interactive plotting.  We're going to try to plot some SDSS spectroscopic data.  In particular, we'll read BOSS/eBOSS [spPlate](https://data.sdss.org/datamodel/files/BOSS_SPECTRO_REDUX/RUN2D/PLATE4/spPlate.html) files that contain the flux *versus* wavelength data and match them up with [spZbest](https://data.sdss.org/datamodel/files/BOSS_SPECTRO_REDUX/RUN2D/PLATE4/RUN1D/spZbest.html) files that contain the best-fit redshift and model.

In the Setup section below, we first load all the necessary code, then in the section Loading Spectra below, we actually start loading and plotting the spectra.

This notebook is inspired by [the Inspector](https://github.com/sbailey/inspector/).

<a class="anchor" id="Disclaimer-&-Attribution"></a>

## Disclaimer & Attribution

If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick *et al.*, "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445
* Data Lab disclaimer: http://datalab.noao.edu/disclaimers.php

## Imports & Setup

In this section we simply load all the code to create the plots.

In [ ]:
# Standard Library
# import os
from io import BytesIO
from getpass import getpass
from sdss_viz_bokeh import load_sdss_spectra

# Authentication
Much of the functionality of Data Lab can be accessed without explicitly logging in (the service then uses an anonymous login). But some capacities, for instance saving the results of your queries to your virtual storage space, require a login (i.e. you will need a registered user account).

If you need to log in to Data Lab, issue this command, and respond according to the instructions:

In [ ]:
# ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
ac.whoAmI()

## Loading Spectra

Use `load_sdss_spectra(spPlate, spZbest)` to load a `spPlate*.fits` file. A matching `spZbest` file may also be provided, but if not, the location of this file can be deduced from the `spPlate` file.

SDSS-specific file information:

In [ ]:
release = 16
run2d = 'v5_13_0'
run1d = run2d
plate = 4055
mjd = 55359
spPlate = f'sdss_dr{release:d}://eboss/spectro/redux/{run2d}/{plate:d}/spPlate-{plate:04d}-{mjd:5d}.fits'

sp = load_sdss_spectra(spPlate)

## Plotting

Start the interactive viewer with the `.plot()` method.

* The large window displays a downsampled spectrum with best fit model overlaid.
* Scanning the mouse over that plot shows a full-resolution zoom of that portion of the spectrum, which is particularly useful for scanning narrow emission lines.
* Try the various tools at the top for panning, zooming, selecting. When the wheel zoom tool is selected (default), using the scroll wheel will zoom in and out. Point the mouse over the x or y axis before scrolling to zoom on just that axis.
* Click on one of the legend entries to turn that item on/off.
* The buttons at the bottom can be used to navigate between targets and perform other tasks:
  * prev / next: go to the previous or next target without recording visual inspection results
  * the other buttons record visual inspection results and advance to the next target:
    - flag: flag this target as having bad data
    - bad: bad data (*e.g.* low-S/N) such that no redshift fitter could get a confident redshift
    - no: the fitted redshift is incorrect
    - maybe: the fitted redshift might be correct, but not confidently
    - yes: the fitted redshift is definitely correct
  * Emission / Absorption Lines: toggle the display of prominent emission or absorption lines.

In [ ]:
sp.plot()

## Visual inspection results

The visual inspection results are stored in a `visual_scan` table which can be inspected and saved. This table includes the scanner's username so that results from different scanners can be cross checked. Tables for different spectra files can also be stacked to create truth tables. Try navigating through a few spectra marking them as flag/bad/no/maybe/yes, then view the scan results table and save it.

In [ ]:
sp.visual_scan

Save the results locally as a FITS file:

In [ ]:
sp.visual_scan.write('scan_results.fits', overwrite=True)

A user can also save the results to their virtual storage VOSpace:

In [ ]:
sc.put('./scan_results.fits', 'vos://scan_results.fits')

We can ensure it has made it to VOSpace by calling the `sc.ls()` function, which lists all files currently in a user's VOSpace:

In [ ]:
listing = sc.ls(name='vos://')
print(listing)

## Navigating through targets

You can use the buttons to go the previous/next targets, or use the `.prev()` and `.next()` functions. *e.g.* to watch a loop over all the targets, you could do the following (scroll back up to watch the display update)

In [ ]:
import time
sp.ispec = 0
for i in range(sp.nspec):
    sp.next()
    time.sleep(0.5)